In [89]:
import pandas as pd
df = pd.read_csv('turnstile_unit_zip.csv')

In [115]:
df[df.isna().any(axis=1)]

,stop_name,line,zipcode,turnstile_station_id,match_method
5,104102ndsts,jz,11418,NaN,NaN
76,23stelyavenue,em,11101,NaN,NaN
96,42ndstportauoritybusterm,ace,10036,NaN,NaN
97,45courthousesq,77,11101,NaN,NaN
106,50st,2,10107,NaN,NaN
...,...,...,...,...,...
450,w4stwashingtonsqlower,bdfm,10011,NaN,NaN
451,w4stwashingtonsqupper,ace,10014,NaN,NaN
458,whitehallst,nr,10004,NaN,NaN
465,woodside61stst,77,11377,NaN,NaN


In [116]:
df1 = pd.read_csv('2020_mta_counts.csv')

In [118]:
df2 = df[['zipcode', 'turnstile_station_id']]

In [130]:
df2.rename(columns={'turnstile_station_id': 'unit'}, inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [131]:
df2.head()

,zipcode,unit
0,10025-4403,r314
1,10029,r180
2,10025,r191
3,11368,r208
4,11419,r354


In [137]:
df1.unit = df1.unit.str.lower()

In [138]:
dfm = pd.merge(df1, df2, on='unit')

In [139]:
dfm.head()

,Unnamed: 0,unit,scp,station,linename,date,entries,day_count,zipcode
0,2773,r003,00-00-00,CYPRESS HILLS,J,2020-03-15,622712,164.0,11208
1,2774,r003,00-00-00,CYPRESS HILLS,J,2020-03-16,623023,311.0,11208
2,2775,r003,00-00-00,CYPRESS HILLS,J,2020-03-17,623294,271.0,11208
3,2776,r003,00-00-00,CYPRESS HILLS,J,2020-03-18,623525,231.0,11208
4,2777,r003,00-00-00,CYPRESS HILLS,J,2020-03-19,623756,231.0,11208


In [152]:
dfm.zipcode[0]

'11208'

In [141]:
dfc = pd.read_csv('covid_curated.csv')

In [156]:
dfc.zipcode = dfc.zipcode.astype(str)

In [157]:
dfc.tail()

,zipcode,positive,prev_positive,daily_new,date
7962,10455,1187,1184.0,3.0,2020-05-17
7963,11233,1180,1173.0,7.0,2020-05-17
7964,11412,1180,1173.0,7.0,2020-05-17
7965,10017,132,130.0,2.0,2020-05-17
7966,11368,4213,4191.0,22.0,2020-05-17


In [145]:
dfc.drop(columns='Unnamed: 0', inplace=True)

In [180]:
df_master = pd.merge(dfm, dfc, on=['date', 'zipcode'])

In [181]:
df_master.head()

,Unnamed: 0,unit,scp,station,linename,date,entries,day_count,zipcode,positive,prev_positive,daily_new
0,2789,r003,00-00-00,CYPRESS HILLS,J,2020-03-31,625201,100.0,11208,350,NaN,NaN
1,2866,r003,00-00-01,CYPRESS HILLS,J,2020-03-31,1974157,56.0,11208,350,NaN,NaN
2,2943,r003,00-00-02,CYPRESS HILLS,J,2020-03-31,4874451,105.0,11208,350,NaN,NaN
3,187510,r200,00-00-00,EUCLID AV,AC,2020-03-31,1172470,291.0,11208,350,NaN,NaN
4,187587,r200,00-00-01,EUCLID AV,AC,2020-03-31,6243919,581.0,11208,350,NaN,NaN


In [182]:
df_master.drop(columns=['Unnamed: 0', 'prev_positive'], inplace=True)

In [194]:
df_master.tail()

,unit,scp,station,linename,date,entries,day_entries,zipcode,positive,day_cases
156380,r463,00-06-06,CANAL ST,JNQRZ6W,2020-05-17,102766562,12.0,10013,245,0.0
156381,r463,00-06-07,CANAL ST,JNQRZ6W,2020-05-17,4943536,8.0,10013,245,0.0
156382,r463,00-06-07,CANAL ST,JNQRZ6W,2020-05-17,4943536,8.0,10013,245,0.0
156383,r463,00-06-08,CANAL ST,JNQRZ6W,2020-05-17,1691542,13.0,10013,245,0.0
156384,r463,00-06-08,CANAL ST,JNQRZ6W,2020-05-17,1691542,13.0,10013,245,0.0


In [ ]:
df_master.rename(columns={'day_count': 'day_entries', 'daily_new': 'day_cases'}, inplace=True)

In [198]:
(df_master
        .sort_values(['day_cases', 'day_entries'])
        .groupby(['station', 'zipcode'])[['day_cases', 'day_entries']].last()
        .sort_values('day_cases', ascending=False).head(10))

,,day_cases,day_entries
station,zipcode,,
ELMHURST AV,11373,412.0,280.0
GRAND-NEWTOWN,11373,412.0,364.0
103 ST-CORONA,11368,404.0,594.0
JUNCTION BLVD,11368,404.0,489.0
111 ST,11368,404.0,460.0
90 ST-ELMHURST,11372,275.0,441.0
82 ST-JACKSON H,11372,275.0,319.0
50 ST,11219,266.0,253.0
FT HAMILTON PKY,11219,266.0,183.0


In [191]:
df_master.sample()

,unit,scp,station,linename,date,entries,day_entries,zipcode,positive,day_cases
71181,r224,00-00-01,CLARK ST,23,2020-04-30,7229357,18.0,11201,520,8.0


In [175]:
df_master.to_csv('mta_and_covid.csv')

In [192]:
len(df_master.station.sort_values().unique())

300